In [ ]:
# Encounter Flow
# 2 Actions of any type per turn
# 1. Check if in combat (follow flow chart for combat), 2. Move to next room, 3. Use Items, 4. Use Abilities, 5. End Turn
# Each room chooses a random event upon 1st entrance, Each room does a unique thing as a static effect while inside the room
# Can loot each room for items a random number of times rolled on 1st entrance
# When moving, noise check to see if monster spawns, if spawned choose from random encounter bag and resolve
# As many small items as you want, only two big items at a time (can be dropped/picked up if needed?)
# Objective is to turn on all generators and set self destruct/rescue people (haven't decided yet)
# Classes and Room identifiers/events in UML
# Enemy types: 4 tiers of increasing difficulty? Or just 4 of 1 that have separate ways to kill
# Need to come back and start by making character + character classes. Then rooms. Then encounters and events. 
# Win condition can be saved for when the idea is more solid. Inspiration will come as I decide how hard this will be to code